In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Загрузка и предобработка данных
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(root='path/to/train/dataset', transform=transform)
test_data = datasets.ImageFolder(root='path/to/test/dataset', transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Определение архитектуры нейронной сети
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)  # 2 класса: человек или не человек

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

# Создание модели и определение функции потерь и оптимизатора
model = CustomNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Вычисление точности на тестовом наборе данных
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

# Сохранение обученной модели
torch.save(model.state_dict(), 'custom_model.pth')


In [36]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Define the dataset and data loader
train_folder = 'D:/Data/Train'
test_folder =  'D:/Data/Test'
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])
train_dataset = datasets.ImageFolder(train_folder, transform=transform)
test_dataset = datasets.ImageFolder(test_folder, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the neural network and define the loss function and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# print(torch.__version__)

# Train the neural network
for epoch in range(15):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print training progress
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{15}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # Evaluate the neural network on the test set
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            scores = model(data)
            _, predicted = torch.max(scores.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')


Epoch [1/15], Batch [0/2962], Loss: 0.6929
Epoch [1/15], Batch [100/2962], Loss: 0.7080
Epoch [1/15], Batch [200/2962], Loss: 0.7202
Epoch [1/15], Batch [300/2962], Loss: 0.7162
Epoch [1/15], Batch [400/2962], Loss: 0.6445
Epoch [1/15], Batch [500/2962], Loss: 0.6423
Epoch [1/15], Batch [600/2962], Loss: 0.5376
Epoch [1/15], Batch [700/2962], Loss: 0.6489
Epoch [1/15], Batch [800/2962], Loss: 0.6349
Epoch [1/15], Batch [900/2962], Loss: 0.5736
Epoch [1/15], Batch [1000/2962], Loss: 0.5756
Epoch [1/15], Batch [1100/2962], Loss: 0.8072
Epoch [1/15], Batch [1200/2962], Loss: 0.6294
Epoch [1/15], Batch [1300/2962], Loss: 0.6172
Epoch [1/15], Batch [1400/2962], Loss: 0.6777
Epoch [1/15], Batch [1500/2962], Loss: 0.7101
Epoch [1/15], Batch [1600/2962], Loss: 0.6234
Epoch [1/15], Batch [1700/2962], Loss: 0.6689
Epoch [1/15], Batch [1800/2962], Loss: 0.6146
Epoch [1/15], Batch [1900/2962], Loss: 0.5933
Epoch [1/15], Batch [2000/2962], Loss: 0.6923
Epoch [1/15], Batch [2100/2962], Loss: 0.5457


In [37]:
torch.save(model.state_dict(), 'person_detection_model_v4_69.pth')
print('Модель создана')

Модель создана
